# Delineation of agricultural fields using SAMGEO
Contemporary agriculture faces many challenges growing population, climate changes, soil degradation and natural disacters. Due to all this challenges precise agriculture (PA) becomes crucial in agricultural fields monitoring, PA can help farmers in monitoring crops quality, soil moisture and can also optimise crops managment. Constant monitoring of agricultural fields is associated with necessity of defiying their boundaries. Manual defiying of crop fields boundaries using GIS tools can be time consuming, therefore application of machine learning models is gaining popularity.

In this notebook I will show process of delineating crop fields boundaries using Segment Anything Model 2 and Samgeo package.

To run this notebook make sure that you are using GPU, in Colab enviroment T4 GPU with 15 GB should be enought. To use fine-tuned SAM 2 models run SAM_Training notebook.




## Install required packages


In [ ]:
# Remove dev null redirection to print output
%pip install geedim &> /dev/null
%pip install segment-geospatial &> /dev/null
%pip install localtileserver &> /dev/null
%pip install PyCRS &> /dev/null

## Imoport packages

In [ ]:
import os
import ee
import geemap
import geedim
import glob
import torch
import gc
import time
from google.colab import drive

## Inicjalization of Google Earth Engine API
Inicjlize Google Earth Engine (GEE) to use their API for data computaions and access to GEE raster databases.
To inicjalize Google Earth Engine (GEE) mak sure that you have created poroject on [google cloud](https://console.cloud.google.com) platform that have GEE API enabled. Change project name to created by you.

In [ ]:
# Inicjalization of Google Earth Engine API
project_name = "agriculture-delineation" #@param
ee.Authenticate()
ee.Initialize(project=project_name)

# Crate map
Map = geemap.Map()


## Import SAMGEO packages
Samgeo packages have to be imported after map declaration, otherwise conflicts with geemap can ocure.

In [ ]:
from samgeo import tms_to_geotiff
from samgeo import SamGeo2
from samgeo import split_raster as sam_split_raster
import samgeo.common as common

## What is SAMGEO
[Segment geospatial (SAMGEO)](https://samgeo.gishub.org/) is Python package for segmentation of satelite images with Segment Anything Model (SAM), it's developd by Dr. Qiusheng Wu. Main reason for Samgeo creation was to facilitate the use of SAM for geospatial data, Samgeo provide a straightforward interface to segment satelite data imagery and export it's results to various formats like vector or raster masks.

In [ ]:
!pip show segment-geospatial

## Define data download functions
Data is downloaded from database using GEE APi for Python. For segmentation purpose data from 3 raster images collection will be used:

*   NAIP
*   Sentinel 2
*   Landsat


### Sentinel 2
Sentinel 2A is European Space Agency optical imaging satellite, it was launched in 2015 as an part of Copernicus Programe. Images collected by Sentinel 2 are multi-spectral and pixels resolution starts from 10 meters. Pixels values are shifted by 10000, in newer images this value is equal 1000. Harmonised collection used in this notebook shifts data in newer scenes to range used in older scenes.

Sentinel 2 image is loaded for GEE enviromet using ee.ImageCollection function, images are selected using given criteriums then pixels with clouds are masked using masking function. Clouds coverage information is store in QA60 band, 10 and 11 bits are related with cloud occurance. From images given bands are loaded and slected as mean value for all images. Pixels values are then divied by 10000 (the Sentinel 2 scale factor). Laslty image is cliped to boundaries of area of intrest.

In [ ]:
def download_sentinel2_image(aoi, start_date, end_date, bands_list, clouds_percentage):
  """
  Download Sentinel 2 image from Google Earth Engine.

  Parametrs:
  aoi: geometry of area of interest, ee.Geometry
  start_date
  """

  def mask_clouds(image):
    """
    Mask clouds on Sentinel 2 image.

    Arguments:
    image: image to be remove clouds from

    Returns
    image: image without clouds
    """

    # Select Sentinel 2 quality band
    qa = image.select("QA60")

    # Dfine mask for clouds (10 bit) and cirruses (bit 11)
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Value 0 - no clouds and cirrues
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask)


  # Download image as a median from collection
  image = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', clouds_percentage)
    .filterDate(start_date, end_date)
    .filterBounds(aoi)
    .map(mask_clouds)
    .select(bands_list)
    .median()
  )

  # Divide image by Sentinel 2 scale factor
  rescale_image = image.divide(10000)

  # Clip image to area of intrest
  clip_rescale_image = rescale_image.clip(aoi)

  return clip_rescale_image



### Landsat 8
Landsat 8 is USA satelite launched in 2013 and collecting data across the world. Landsat 8 measures reflected electromagntic wave information in various bands using buildin sensors, base spatial resolution of pixels is 30 meters.

Loading process of Landsat 8 image starts by defiying function applying scale factor to pixels values. Landsat optical bands pixels are multiply by 0.0000275 and -0.2 is added. To ensure image quality clouds masking function is needed, Landsat images comes with special quality ansurance (QA) band in which information of cloud occurance is stored. While image is loaded for GEE boath scale factor and cloud masking functions are used, then give image bands are selected as mean value of pixels. At the end whole image is cliped to area of intrest.

In [ ]:
def download_landsat8_image(aoi, start_date, end_date, bands_list, clouds_percentage):
  """
  Download Landsat 8 image from Google Earth Engine.

  Parametrs:
  aoi: geometry of area of interest, ee.Geometry
  start_date: date to start search for images
  end_date: date to end search for images
  bands_list: list of bands to be downloaded
  clouds_percentage: percentage of clouds in image
  """

  def apply_scale_factors(image):
    """
    Scale image by Landsat 8 bands scales.

    image: landsat image, ee.image
    """

    # Scale image by Landsat 8 bands scales.
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    # Add scaled bands to iamge
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

  def mask_clouds(image):
    """
    Mask clouds and shadows on Landsat 8 image using QA_PIXEL band.

    image: landsat image, ee.image
    """
    qa = image.select("QA_PIXEL")

    # Define masks for clouds (bit 3) and their shadows (bit 4)
    cloud_bit_mask = 1 << 3
    cloud_shadow_bit_mask = 1 << 4

    # Value 0: no clouds and shadows
    cloud_mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0))

    return image.updateMask(cloud_mask)


  # Landsat 8 Collection 2, Tier 1, Level-2
  dataset = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(start_date, end_date)
    .filterBounds(aoi)
    .map(mask_clouds)
    .map(apply_scale_factors)
    .select(bands_list)
    .median()
  )

  # Clip image to aoi
  clip_rescale_image = dataset.clip(aoi)

  return clip_rescale_image


### NAIP
Image from USA National Agriculture Imagery Program collecting high resolution Airborne images. Images since 2018 comes in 0.6 meters resoultion and include 4 bands (RGBN), before 2018 data was collected only in reslution of 1 or 2 meters with RGB bands. NAIP images quailty is controled by automate and visuall methods.

NAIP data is loaded from GEE enviroment as median value, image is then cliped to area of intrest boundaries and converted to unsigned int 8.

In [ ]:
def download_naip_image(aoi, start_date, end_date, bands_list):
  """
  Download USA National Agriculture Imagery Program (NAIP) image from Google Earth Engine.

  aoi: geometry of area of interest, ee.Geometry
  start_date: date to start search for images, min 2002-06-15
  end_date: date to end search for images, max 2023-11-01
  bands_list: list of bands to be downloaded, available ["R", "G", "B", "N"]
  """

  # NAIP
  dataset = (ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterDate(start_date, end_date)
    .filterBounds(aoi)
    .select(bands_list)
    .median()
  )

  # Clip image to aoi
  clip_rescale_image = dataset.clip(aoi)

  cast_image = clip_rescale_image.toUint8()

  return cast_image


In [ ]:
#@title save image function
def save_ee_image(image, file_path, scale, aoi, convert_to_int=True):
  """
  Save ee.image as GeoTIFF.

  image: ee.image to be downloaded
  file_path: path to save imge
  scale: scale to rescale image
  aoi: geometry of area of interest, ee.Geometry
  convert_to_int: convert image to uint8
  """

  # Get image crs code
  crs = image.select(0).projection().crs().getInfo()

  # Convrt image to uint8 if nesesary
  if convert_to_int:
    image = image.multiply(255).uint8()

  geemap.download_ee_image(
    image = image,
    filename = file_path + ".tif",
    scale = scale,
    region = aoi,
    crs = crs
  )

In [ ]:
#@title Split raster function
def split_raster(file_path, out_dir, tile_size):
  """
  Split image into smaller tiles.

  file_path: path to image, str
  out_dir: path to save tiles, str
  tile_size: size of tile in pixels, tuple
  """

  # Split image into smaller tiles
  split_raster(
    filename = file_path,
    out_dir = out_dir,
    tile_size = tile_size
  )

## Area of intrest
Chosen area of intrest for this analysis is located in Illinois, 30 km west of Chicago. AOI represents part of Illinois agricuture region, aoi covers aproximetly 139 km<sup>2</sup>.

In [ ]:
aoi_usa = ee.Geometry.Polygon([
    [[-88.95, 41.8],
    [-88.80, 41.8],
    [-88.80, 41.9],
    [-88.95, 41.9],
    [-88.95, 41.8]]
]);



## Download data
To download data first required directories are created. Directory have following scheme:
```
content
├───naip
│   ├───images
│   ├───masks
│   └───tiles
...
```

Once direstories are created data from all sources is downloaded. Date filter is set from 2020-01-01 to 2021-12-31, all images are selected as RGB bands and max cloud coverage is set to 20%. NAIP images are cut to 3000 px tiles, Sentinel to 500 px and Landsat to 1000 px.

In [ ]:
os.makedirs("sentinel/images", exist_ok=True)
os.makedirs("sentinel/masks", exist_ok=True)
os.makedirs("sentinel/tiles", exist_ok=True)

os.makedirs("landsat/images", exist_ok=True)
os.makedirs("landsat/masks", exist_ok=True)
os.makedirs("landsat/tiles", exist_ok=True)

os.makedirs("naip/images", exist_ok=True)
os.makedirs("naip/masks", exist_ok=True)
os.makedirs("naip/tiles", exist_ok=True)

In [ ]:
#@title NAIP
image_naip_rgb = download_naip_image(aoi_usa, '2020-01-01', '2021-12-31', ['R', 'G', 'B'])

save_ee_image(image_naip_rgb, "naip/images/rgb_usa", scale=0.6, aoi=aoi_usa, convert_to_int=False)
sam_split_raster(filename = "naip/images/rgb_usa.tif", out_dir = "naip/tiles/rgb_usa", tile_size = (3000, 3000), overlap=True)

In [ ]:
#@title Sentinel
image_sentinel_rgb = download_sentinel2_image(aoi_usa, '2020-01-01', '2021-12-31', ['B4', 'B3', 'B2'], 20)

save_ee_image(image_sentinel_rgb, "sentinel/images/rgb_usa", 10, aoi_usa)
sam_split_raster(filename = "sentinel/images/rgb_usa.tif", out_dir = "sentinel/tiles/rgb_usa", tile_size = (500, 500), overlap=True)

In [ ]:
#@title Landsat
image_landsat_rgb = download_landsat8_image(aoi_usa, '2020-01-01', '2021-12-31', ['SR_B4', 'SR_B3', 'SR_B2'], 20)

save_ee_image(image_landsat_rgb, "landsat/images/rgb_usa", 30, aoi_usa)
sam_split_raster(filename = "landsat/images/rgb_usa.tif", out_dir = "landsat/tiles/rgb_usa", tile_size = (1000, 1000), overlap=True)


## Images visualization

In [ ]:
vis_params= {
  'min': 0,
  'max': 1,
  'gamma': 2
}


vis_params_polygon = {
    "color": "000000",
    "fillColor": "4d486a",
    "colorOpacity": 1,
    "width": 2,
    "lineType": "solid",
    "fillColorOpacity": 1,
}

In [ ]:
Map.addLayer(image_naip_rgb, {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}, 'NAIP_rgb')
Map.addLayer(image_sentinel_rgb, vis_params, 'Sentinel_rgb')
Map.addLayer(image_landsat_rgb, vis_params, 'Landsat_rgb')

Map.centerObject(aoi_usa, 12)
Map

## Segmentation and delineation



First SAM 2 segmentation and delineation helper function is defined. This function loads all images paths from given directories and for every of them generates mask. If mask is not empty it will be saved under output path, then GPU memory is cleaned. Once all images are procesed segmentation masks are merged and vertorised into seperated files.

In [ ]:
from tqdm import tqdm

def sam_generate_batch(sam, in_dir, out_dir, merge=True, vectorize=False):
  """
  Segments all images located in given directory, final results can be merged and vectorised.

  sam: sam 2 model to be used
  in_dir: path where images are located
  out_dir: path to store results
  merge: whether to merge masks
  vectorize: whether to vectorize masks
  """

  # Get all files in directory
  tif_files = glob.glob(os.path.join(in_dir, '*.tif'))
  # Make output directoryl
  os.makedirs(out_dir, exist_ok=True)


  for i, file in enumerate(tqdm(tif_files, desc="Processing TIFF files")):
    #  Generate mask
    sam.generate(file)

    # Check if mask is area is not 0
    if len(sam.masks) > 0 and sam.masks[0]['segmentation'].size > 0:
      # Save mask
      if out_dir is not None:
        sam.save_masks(output= out_dir + "/mask_" + str(i) + ".tif")
      else:
        sam.save_masks(output= "mask_" + str(i) + ".tif")

    # Clean GPU memory
    gc.collect()
    torch.cuda.empty_cache()

  # Merge mask
  if merge:
    print(f"Merging masks")
    common.merge_rasters(out_dir, output= out_dir + "/merged_masks.tif")

  # Vectroise mask
  if vectorize:
    print(f"Vectorizing masks")
    common.raster_to_vector(out_dir + "/merged_masks.tif", output= out_dir + "/merged_masks.shp")

  # Clean GPU memory
  gc.collect()
  torch.cuda.empty_cache()


Connect to drive to load fine-tuned model weights and create directories to store results data.

In [ ]:
# Connect to drive
drive.mount('/content/gdrive', force_remount=True)

# Load finetune model weights
finetuned_tiny = torch.load("/content/gdrive/MyDrive/SAM_trained_models/fine_tuned_sam2_tiny_epoch_14.torch")
finetuned_small = torch.load("/content/gdrive/MyDrive/SAM_trained_models/fine_tuned_sam2_small_epoch_14.torch")
finetuned_base_plus = torch.load("/content/gdrive/MyDrive/SAM_trained_models/fine_tuned_sam2_base_epoch_14.torch")
finetuned_large = torch.load("/content/gdrive/MyDrive/SAM_trained_models/fine_tuned_sam2_large_epoch_14.torch")

# Make directories to store results
!mkdir -p /content/gdrive/MyDrive/Agriculture_tif
!mkdir -p /content/gdrive/MyDrive/Agriculture_shp


### Setup Samgeo




Choose SAM 2 pretrained model checkpoint or fine-tuned. Then segmentation function parameters are show below.

*   points_per_side - The number of points to be sampled along one side of the image. The total number of points is points_per_side**2. If None, 'point_grids' must provide explicit point sampling.
*   points_per_batch - Sets the number of points run simultaneously by the model. Higher numbers may be faster but use more GPU memory.
*   pred_iou_tresh - A filtering threshold in [0,1], using the model's predicted mask quality.
*   stabilty_score_tresh - A filtering threshold in [0,1], using the stability of the mask under changes to the cutoff used to binarize the model's mask predictions.
*   stability_score_offset - The amount to shift the cutoff when calculated the stability score.
*   crop_n_layers - If >0, mask prediction will be run again on crops of the image. Sets the number of layers to run, where each layer has 2**i_layer number of image crops.
*   box_nms_tresh - The box IoU cutoff used by non-maximal suppression to filter duplicate masks.
*   crop_n_points_downscale_factor - The number of points-per-side sampled in layer
*   use_m2m - Whether to add a one step refinement using previous mask predictions.



In [ ]:
# model_id = "sam2-hiera-large" #@param ["sam2-hiera-tiny", "sam2-hiera-small", "sam2-hiera-base-plus", "sam2-hiera-large"]
model = "base_plus" #@param  ["tiny", "small", "base_plus", "large"]
use_finetuned_model = True #@param {type:"boolean"}

model_id = "sam2-hiera-" + model.replace("_", "-")

Sam2 = SamGeo2(
    model_id=model_id,
    automatic=True,
    output_mode='binary_mask',
    points_per_side=32,
    points_per_batch=64,
    pred_iou_thresh=0.7,
    stability_score_thresh=0.7,
    stability_score_offset=0.7,
    crop_n_layers=1,
    box_nms_thresh=0.7,
    crop_n_points_downscale_factor=2,
    min_mask_region_area=50.0,
    use_m2m=True
  )

if use_finetuned_model:
  weights = "finetuned_" + model
  weights = eval(weights)
  res = Sam2.mask_generator.predictor.model.load_state_dict(weights)
  print(res)

### Segment images

In [ ]:
image = "landsat" #@param ["naip", "landsat", "sentinel"]
copy_to_drive = True #@param {type:"boolean"}
in_dir = image + "/tiles/rgb_usa"
if use_finetuned_model:
  model_type = "finetuned"
else:
  model_type = "pretrained"

out_dir = image + "/masks/rgb_" + model_type + "_" + model


start_time = time.time()

# Segment images
sam_generate_batch(
    sam=Sam2,
    in_dir=in_dir,
    out_dir=out_dir,
    vectorize=True)

end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")

if copy_to_drive:

  colab_file_path = out_dir + "/merged_masks"
  drive_tif_file_path = '/content/gdrive/MyDrive/Agriculture_tif/' + image + "_" + model_type + "_" + model + ".tif"
  drive_shp_file_path = "/content/gdrive/MyDrive/Agriculture_shp/" + image + "_" + model_type + "_" + model

  # Copy the tif file
  !cp {colab_file_path + ".tif"} {drive_tif_file_path}

  #Copy the shp files
  !cp {colab_file_path + ".cpg"} {drive_shp_file_path + ".cpg"}
  !cp {colab_file_path + ".dbf"} {drive_shp_file_path + ".dbf"}
  !cp {colab_file_path + ".prj"} {drive_shp_file_path + ".prj"}
  !cp {colab_file_path + ".shp"} {drive_shp_file_path + ".shp"}
  !cp {colab_file_path + ".shx"} {drive_shp_file_path + ".shx"}


## Validate delineation
To validate agricutural fields dlineation [ESA cropland dataset](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCereal_2021_MARKERS_v100) will be used. Dataset conatins image representation of agricultural fields on USA trritory. Each pixel have 10m resoultion and can represent active or noactive fields. Since the goal is to delineate all fields both pixels values will be mrged into new binary layer and vactorise.



In [ ]:
def download_esa_cropland(aoi, start_date, end_date, filename, scale):
  """
  Download ESA cropland dataset from Google Earth Engine.

  aoi: geometry of area of interest, ee.Geometry
  start_date: date to start search for images
  end_date: date to end search for images
  filename: path to save image, str
  scale: scale to rescale image
  """

  # Load dataset as sum of active and inactive croplands
  dataset = (ee.ImageCollection('ESA/WorldCereal/2021/MARKERS/v100')
    .filterDate(start_date, end_date)
    .sum()
  )

  dataset = dataset.clip(aoi)

  cropLandcover = dataset.select('classification')

  # Vectorise cropland image
  vectorised_croplandcover = cropLandcover.reduceToVectors(
        geometryType='polygon',
        reducer=ee.Reducer.countEvery(),
        scale=scale,
        maxPixels=1e13,
        geometry=aoi
    )

  # Save cropland image
  save_ee_image(cropLandcover, filename, scale=scale, aoi=aoi, convert_to_int=True)

  # Save cropland shapefile
  geemap.ee_to_shp(vectorised_croplandcover, filename + ".shp")


Define function to measure segmentation accuracy using intersection over union (IoU) metric also known as Jaccard's Index. Jaccard's Index formula:

$J[A, B] = \frac{|A \cap B|}  {|A \cup B|}$.

The function loads two tif images and computes number of intersection and union pixels, next returns division result.

In [ ]:
import numpy as np
from PIL import Image

# Raise limit of image pixels number
Image.MAX_IMAGE_PIXELS = 600000000

def calculate_iou(image1_path, image2_path):
    """
    Calculate intersection over union (IoU) between two binary images in TIFF format.

    Parameters:
    - image1_path: Path to the first binary TIFF image.
    - image2_path: Path to the second binary TIFF image.

    Returns:
    - iou: The IoU value between the two images.
    """

    # Load TIFF images using PIL
    image1 = np.array(Image.open(image1_path))
    image2 = np.array(Image.open(image2_path))

    # Ensure both images have the same shape
    if image1.shape != image2.shape:
        print(image1.shape)
        print(image2.shape)
        raise ValueError("The input images must have the same dimensions.")

    # Convert images to binary to make sure that values are correct
    binary_image1 = image1 > 0
    binary_image2 = image2 > 0

    # Calculate intersection and union
    intersection = np.sum(np.logical_and(binary_image1, binary_image2))  # Pixels where both are 1
    union = np.sum(np.logical_or(binary_image1, binary_image2))  # Pixels where either is 1

    # Avoid division by zero
    if union == 0:
        return 0.0

    # Calculate IoU
    iou = intersection / union

    return iou

Download ESA cropland data in scales corresponding to segmented images, date is also filtered to be same as them.

In [ ]:
download_esa_cropland(aoi_usa, '2020-03-01', '2021-04-20', "esa_croplands_06", scale=0.6)
download_esa_cropland(aoi_usa, '2020-03-01', '2021-04-20', "esa_croplands_10", scale=10)
download_esa_cropland(aoi_usa, '2020-03-01', '2021-04-20', "esa_croplands_30", scale=30)

In [ ]:
import matplotlib.pyplot as plt

# SAM2 models names
models = ['Tiny', 'Small', 'Base+', 'Large']

# SAM2 models accuracy scores
pretrained = []
finetuned = []

pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_pretrained_tiny.tif", "/content/esa_croplands_06.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_pretrained_small.tif", "/content/esa_croplands_06.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_pretrained_base_plus.tif", "/content/esa_croplands_06.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_pretrained_large.tif", "/content/esa_croplands_06.tif"))

finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_finetuned_tiny.tif", "/content/esa_croplands_06.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_finetuned_small.tif", "/content/esa_croplands_06.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_finetuned_base_plus.tif", "/content/esa_croplands_06.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/naip_finetuned_large.tif", "/content/esa_croplands_06.tif"))

print(pretrained)
print(finetuned)

# X axis names arrange
x = np.arange(len(models))

width = 0.35

# Create plot
plt.bar(x - width/2, pretrained, width=width, label='Pre-Trained', color='#4C72BB')
plt.bar(x + width/2, finetuned, width=width, label='Fine-Tuned', color='#55BB68')

# Add plot descriptions
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy of SAM 2 models with NAIP images')
plt.xticks(x, models)
plt.legend()

plt.savefig("NAIP_accuracy.png", dpi=300)

plt.tight_layout()
plt.show()


In [ ]:
# SAM2 models names
models = ['Tiny', 'Small', 'Base+', 'Large']

# SAM2 models accuracy scores
pretrained = []
finetuned = []

pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_pretrained_tiny.tif", "/content/esa_croplands_10.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_pretrained_small.tif", "/content/esa_croplands_10.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_pretrained_base_plus.tif", "/content/esa_croplands_10.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_pretrained_large.tif", "/content/esa_croplands_10.tif"))

finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_finetuned_tiny.tif", "/content/esa_croplands_10.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_finetuned_small.tif", "/content/esa_croplands_10.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_finetuned_base_plus.tif", "/content/esa_croplands_10.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/sentinel_finetuned_large.tif", "/content/esa_croplands_10.tif"))

print(pretrained)
print(finetuned)

# X axis names arrange
x = np.arange(len(models))

width = 0.35

# Create plot
plt.bar(x - width/2, pretrained, width=width, label='Pre-Trained', color='#4C72BB')
plt.bar(x + width/2, finetuned, width=width, label='Fine-Tuned', color='#55BB68')

# Add plot descriptions
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy of SAM 2 models with Sentinel images')
plt.xticks(x, models)
plt.legend()

plt.savefig("Sentinel_accuracy.png", dpi=300)

plt.tight_layout()
plt.show()


In [ ]:
# SAM2 models names
models = ['Tiny', 'Small', 'Base+', 'Large']
import matplotlib.pyplot as plt
# SAM2 models accuracy scores
pretrained = []
finetuned = []

pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_pretrained_tiny.tif", "/content/esa_croplands_30.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_pretrained_small.tif", "/content/esa_croplands_30.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_pretrained_base_plus.tif", "/content/esa_croplands_30.tif"))
pretrained.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_pretrained_large.tif", "/content/esa_croplands_30.tif"))

finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_finetuned_tiny.tif", "/content/esa_croplands_30.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_finetuned_small.tif", "/content/esa_croplands_30.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_finetuned_base_plus.tif", "/content/esa_croplands_30.tif"))
finetuned.append(calculate_iou("/content/gdrive/MyDrive/Agriculture_tif/landsat_finetuned_large.tif", "/content/esa_croplands_30.tif"))

print(pretrained)
print(finetuned)

# X axis names arrange
x = np.arange(len(models))

width = 0.35

# Create plot
plt.bar(x - width/2, pretrained, width=width, label='Pre-Trained', color='#4C72BB')
plt.bar(x + width/2, finetuned, width=width, label='Fine-Tuned', color='#55BB68')

# Add plot descriptions
plt.xlabel('Models')
plt.ylabel('Accuracy')
plt.title('Accuracy of SAM 2 models with Landsat images')
plt.xticks(x, models)
plt.legend()

plt.savefig("Landsat_accuracy.png", dpi=300)

plt.tight_layout()
plt.show()
